In [1]:
import tkp.db
from tkp.db.model import Varmetric
from tkp.db.model import Runningcatalog
from tkp.db.model import RunningcatalogFlux
from tkp.db.model import Newsource
from tkp.db.model import Image
from tkp.db.model import Extractedsource
from sqlalchemy import *
from sqlalchemy.orm import relationship

                ### Need to download tkp
            
import scipy as sp
import numpy as np
import pandas as pd
import sqlalchemy
import random
import numpy as np
import os
from scipy.stats import norm

import logging
import sys
from dblogin import * # This file contains all the variables required to connect to the database
from database_tools import dbtools
from tools import tools       ###Use weblib instead
from plotting import plot_varib_params as pltvp     ###Can't find this package
import matplotlib
import matplotlib.pyplot as plt
import pylab
from matplotlib.ticker import NullFormatter
from matplotlib.font_manager import FontProperties
import seaborn as sns


                ###dblogin, database_tools, tools, plotting are packages I couldn't download

from machine_learning import generic_tools
from machine_learning import MLtests

logging.basicConfig(level=logging.INFO)
query_loglevel = logging.WARNING  # Set to INFO to see queries, otherwise WARNING
sys.path.append('../')
pylab.rcParams['legend.loc'] = 'best'

ContextualVersionConflict: (scipy 1.4.1 (/Users/ujwalkumar/anaconda3/lib/python3.7/site-packages), Requirement.parse('scipy<1.3.0,>=0.7.0'), {'tkp'})

In [5]:
engine = 'postgresql'
host = 'http://banana.transientskp.org/r3'
port = ''
user = 'lofartkp'
password = 'grs1915'

def access(engine,host,port,user,password,database):
    # Access the database using sqlalchemy
    db = tkp.db.Database(engine=engine, host=host, port=port,
                     user=user, password=password, database=database)
    db.connect()
    session = db.Session()
    print ('connected!')
    return session

def GetVarParams(session,dataset_id):
    # Returns all the variability parameters for sources in a given dataset
    VarParams = session.query(Varmetric,Runningcatalog).select_from(join(Varmetric,Runningcatalog)).filter(Runningcatalog.dataset_id == dataset_id).all()
    return VarParams

def GetTransDataForML(session,dataset_id):
    transients = (session.query(Newsource,Runningcatalog)
                      .select_from(join(Newsource,Runningcatalog))
                      .filter(Runningcatalog.dataset_id == dataset_id)
                     .all())
    return transients

def GetRuncatDataForML(session,dataset_id):
    transients = (session.query(Runningcatalog)
                      .join(Varmetric)
                      .join(RunningcatalogFlux)
#                      .select_from(join(Runningcatalog))
                     .filter(Runningcatalog.dataset_id == dataset_id)
                     .all())
    return transients

SyntaxError: invalid syntax (<ipython-input-5-25bb75aa89f2>, line 3)

In [6]:
### CONNECTING TO DATABASE

# The input database, dataset and thresholds
dataset_id = 9
database = 'AR_R4'
websiteURL = 'http://banana.transientskp.org/r4/vlo_'+database+'/runningcatalog/'

path='ml_csv_files/'
stableData = path+'stable_sources.csv'
 
# Connect to the database and run the queries
session = dbtools.access(engine,host,port,user,password,database)

transients = dbtools.GetTransDataForML(session,dataset_id)
transrcs = [[transients[i].Newsource.trigger_xtrsrc.image.detection_thresh,
                 transients[i].Newsource.trigger_xtrsrc.f_int,
                 transients[i].Newsource.newsource_type,
                 transients[i].Newsource.trigger_xtrsrc.image.rms_max,
                 transients[i].Newsource.trigger_xtrsrc.image.rms_min,
                 transients[i].Runningcatalog.id,
                 ] for i in range(len(transients))]


Runcats = dbtools.GetRuncatDataForML(session,dataset_id)
runcats = [[Runcats[i].xtrsrc.image.band.freq_central,
                Runcats[i].dataset.id,
                Runcats[i].varmetric.eta_int,
                Runcats[i].xtrsrc.extract_type,
                Runcats[i].datapoints,
                Runcats[i].xtrsrc.f_int,
                Runcats[i].xtrsrc.f_int_err,
                Runcats[i].xtrsrc.image.freq_eff,
                Runcats[i].id,
                Runcats[i].xtrsrc.image.tau_time,
                Runcats[i].xtrsrc.image.taustart_ts,
                Runcats[i].varmetric.v_int,
                Runcats[i].wm_decl,
                Runcats[i].wm_ra,
                Runcats[i].xtrsrc.id
             ] for i in range(len(Runcats))]


frequencies, srcs = tools.read_src_lc(runcats)

print (runcats[-1])
print (frequencies)

trans_data = tools.collate_trans_data(srcs,frequencies,transrcs)


print (trans_data[-1])

output3 = open('ds'+str(dataset_id)+'_trans_data.txt','w')
output3.write('#Runcat, eta, V, flux, fluxrat, freq, dpts, RA, Dec, ttype, maxRmsSigma, minRmsSigma, detectionThreshold  \n')
for x in range(len(trans_data)):
    string='%s' % ','.join(str(val) for val in trans_data[x])
    output3.write(string+'\n')
output3.close()
print ('Data extracted and saved')

NameError: name 'dbtools' is not defined

In [ ]:
### Defined functions

def trial_data(args):
    # Find the precision and recall for a given pair of thresholds
    data,sigma1,sigma2 = args

    # Sort data into transient and non-transient
    xvals = [float(x[0]) for x in data if float(x[-1]) != 0.]
    yvals = [float(x[1]) for x in data if float(x[-1]) != 0.]
    xstable = [float(x[0]) for x in data if float(x[-1]) == 0.]
    ystable = [float(x[1]) for x in data if float(x[-1]) == 0.]

    # Find the thresholds for a given sigma, by fitting data with a Gaussian model
    sigcutx,paramx,range_x = generic_tools.get_sigcut([float(x[0]) for x in data if float(x[-1]) == 0.],sigma1)
    sigcuty,paramy,range_y = generic_tools.get_sigcut([float(x[1]) for x in data if float(x[-1]) == 0.],sigma2)

    # Count up the different numbers of tn, tp, fp, fn
    fp=len([z for z in range(len(xstable)) if (xstable[z]>sigcutx and ystable[z]>sigcuty)]) # False Positive
    tn=len([z for z in range(len(xstable)) if (xstable[z]<sigcutx or ystable[z]<sigcuty)]) # True Negative
    tp=len([z for z in range(len(xvals)) if (xvals[z]>sigcutx and yvals[z]>sigcuty)]) # True Positive
    fn=len([z for z in range(len(xvals)) if (xvals[z]<sigcutx or yvals[z]<sigcuty)]) # False Negative
    
    # Use these values to calculate the precision and recall values
    precision, recall = generic_tools.precision_and_recall(tp,fp,fn)
    return [sigma1, sigma2, precision, recall]

def multiple_trials(data,filename):
    # Find the precision and recall for all combinations of the sigma thresholds
    sigmas=np.arange(0.,3.5,(3.5/500.))
    pool = Pool(processes=20)              # start 20 worker processes
    inputs = range(2)
    
    # Loop through all the trial sigmas and on multiple workers, then append to a file
    for sigma1 in sigmas:
        sigma_data = pool.map(trial_data, [(data,sigma1,sigma2) for sigma2 in sigmas])
        with open(filename,'a') as f_handle:
            np.savetxt(f_handle,sigma_data)
        f_handle.close()
    pool.close() # Close the worker processes once training is complete
    return

def tests(args):
    # Test multiple input precision and recall values to check out if we are meeting and exceeding the input parameters
    xi,yi,zi1,zi2, data, xvals, yvals, xstable, ystable, precis, recall = args

    # Find the combination of x and y which is closest to the two thresholds
    combinations=[[xi[a][b],yi[a][b],zi1[a][b],zi2[a][b]] for a in range(len(zi1)) for 
                  b in range(len(zi1[0])) if zi1[a][b]>=precis]
    ID=np.array([((a[2]-precis)**2. + (a[3]-recall)**2.) for a in combinations]).argmin()
    above_thresh_sigma=combinations[ID]
    
    # Find the thresholds for these sigmas, by fitting the observed data with a Gaussian model
    sigcutx,paramx,range_x = generic_tools.get_sigcut([float(x[0]) for x in data],above_thresh_sigma[0])
    sigcuty,paramy,range_y = generic_tools.get_sigcut([float(x[1]) for x in data],above_thresh_sigma[1])

    # Count up the different numbers of tp, fp, fn
    fp=len([z for z in range(len(xstable)) if (xstable[z]>sigcutx and ystable[z]>sigcuty)]) # False Positive
    tp=len([z for z in range(len(xvals)) if (xvals[z]>sigcutx and yvals[z]>sigcuty)]) # True Positive
    fn=len([z for z in range(len(xvals)) if (xvals[z]<sigcutx or yvals[z]<sigcuty)]) # False Negative
    
    # Use these values to calculate the precision and recall values obtained with the trained threshold.
    # If the test is successful, the outputs should meet or exceed the input parameters.
    results1, results2 = generic_tools.precision_and_recall(tp,fp,fn)

    return [precis, recall, results1, results2]

def check_method_works(xi,yi,zi1,zi2, data,above_thresh_sigma,path):
    # Multiple trials using input precisions and recalls between 0 and 1 are conducted to confirm that this method provides the input precision and recall
    trials = np.arange(0.0,1.,1./500.)
    trials2 = np.arange(0.0,1.,1./500.)
    Z1=[]
    Z2=[]
    X=[]
    Y=[]

    print "Producing anomaly detection test data"
    if not os.path.exists(path+'anomaly_test_data.txt'): # Only run the test if the output data are not already available
        open(path+'anomaly_test_data.txt','w').close()
        pool = Pool(processes=20)              # start 20 worker processes
        inputs = range(2)

        # Sort data into variable and non-variable
        xvals = [float(x[0]) for x in data if float(x[-1]) != 0.]
        yvals = [float(x[1]) for x in data if float(x[-1]) != 0.]
        xstable = [float(x[0]) for x in data if float(x[-1]) == 0.]
        ystable = [float(x[1]) for x in data if float(x[-1]) == 0.]

        # Run through each of the precision and recall trials and calculate the output precision and recalls
        for precis in trials:
            test_data = pool.map(tests, [(xi,yi,zi1,zi2, data, xvals, yvals, xstable, ystable, precis, recall) for recall in trials2])
            with open(path+'anomaly_test_data.txt','a') as f_handle: # append data to a file
                np.savetxt(f_handle,test_data)
            f_handle.close()
        pool.close() # close all the workers

    # sort the data for plotting
    test_data = np.genfromtxt(path+'anomaly_test_data.txt')
    X = [x[0] for x in test_data] # input precision
    Y = [x[1] for x in test_data] # input recall
    Z1 = [(x[2]-x[0]) for x in test_data] # output precision
    Z2 = [(x[3]-x[1]) for x in test_data] # output precision
    
    print("Anomaly Detection test data collated, now gridding")
    xi,yi = np.mgrid[0:1:1000j, 0:1:1000j]
    zi1 = griddata((X, Y), Z1, (xi, yi), method='cubic')
    zi2 = griddata((X, Y), Z2, (xi, yi), method='cubic')
    # Plot results
    # Settings
    nullfmt   = NullFormatter()         # no labels
    fig = plt.figure(1,figsize=(5,10))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    cax = fig.add_axes([0.1, 0.95, 0.8, 0.03])
    fig.subplots_adjust(hspace = .001, wspace = 0.001)
    levels=np.arange(-0.2,0.25,0.05)
    ax2.set_xlabel(r'Input Precision', fontsize=24)
    ax1.set_ylabel(r'Input Recall', fontsize=24)
    ax2.set_ylabel(r'Input Recall', fontsize=24)
    fontP = FontProperties()
    fontP.set_size('x-small')
    ax1.set_xlim([0.0,1.0])
    ax2.set_xlim([0.0,1.0])
    ax1.set_ylim([0.0,0.999])
    ax2.set_ylim([0.0,0.999])
    ax1.xaxis.set_major_formatter(nullfmt)
    ax2.set_xlim( ax1.get_xlim() )
    ax1.text(2.5, 3.5, 'Precision', bbox=dict(facecolor='white'), fontsize=24)
    ax2.text(2.5, 3.5, 'Recall', bbox=dict(facecolor='white'), fontsize=24)
    CS1 = ax1.contourf(xi,yi,zi1,levels, cmap=plt.get_cmap('RdBu'),alpha=1,extend='both')
    CS2 = ax2.contourf(xi,yi,zi2,levels, cmap=plt.get_cmap('RdBu'),alpha=1,extend='both')
    fig.colorbar(CS2, cax, orientation='horizontal')
    ax1.axvline(x=above_thresh_sigma[0], linewidth=2, color='k', linestyle='--')
    ax2.axvline(x=above_thresh_sigma[0], linewidth=2, color='k', linestyle='--')
    ax1.axhline(y=above_thresh_sigma[1], linewidth=2, color='k', linestyle='--')
    ax2.axhline(y=above_thresh_sigma[1], linewidth=2, color='k', linestyle='--')
    plt.savefig(path+'sim_check_precisions_and_recalls.png')
    plt.close()
    return

def find_best_sigmas(precis_thresh,recall_thresh,data,tests, data2,plots,path):
    # Gridding the data for precision and recall
    X=[float(x[0]) for x in data]
    Y=[float(x[1]) for x in data]
    Z1=[float(x[2]) for x in data]
    Z2=[float(x[3]) for x in data]
    xi,yi = np.mgrid[0:3.5:1000j, 0:3.5:1000j]
    zi1 = griddata((X, Y), Z1, (xi, yi), method='cubic', fill_value=1.)
    zi2 = griddata((X, Y), Z2, (xi, yi), method='cubic', fill_value=0.)

    # Find the combination of x and y which is closest to the two thresholds
    combinations=[[xi[a][b],yi[a][b],zi1[a][b],zi2[a][b]] for a in range(len(zi1)) for b in range(len(zi1[0])) if zi1[a][b]>=precis_thresh]
    ID=np.array([((a[2]-precis_thresh)**2. + (a[3]-recall_thresh)**2.) for a in combinations]).argmin()
    above_thresh_sigma=combinations[ID]

    if plots:
        # Plot results
        # Settings
        nullfmt   = NullFormatter()         # no labels
        fig = plt.figure(1,figsize=(5,10))
        ax1 = fig.add_subplot(211)
        ax2 = fig.add_subplot(212)
        cax = fig.add_axes([0.1, 0.95, 0.8, 0.03])
        fig.subplots_adjust(hspace = .001, wspace = 0.001)
        levels=np.arange(0.0,1.1,0.1)
        levels2=[0.1, 0.3, 0.5, 0.7, 0.9]
        ax2.set_xlabel(r'$\sigma$ threshold ($\eta_{\nu}$)', fontsize=18)
        ax1.set_ylabel(r'$\sigma$ threshold ($V_{\nu}$)', fontsize=18)
        ax2.set_ylabel(r'$\sigma$ threshold ($V_{\nu}$)', fontsize=18)
        fontP = FontProperties()
        fontP.set_size('x-small')
        ax1.set_xlim([0.0,3.5])
        ax2.set_xlim([0.0,3.5])
        ax1.set_ylim([0.0,3.49])
        ax2.set_ylim([0.0,3.49])
        ax1.xaxis.set_major_formatter(nullfmt)
        ax2.set_xlim( ax1.get_xlim() )
        ax1.text(2.5, 3.0, 'Precision', bbox=dict(facecolor='white'), fontsize=18)
        ax2.text(2.5, 3.0, 'Recall', bbox=dict(facecolor='white'), fontsize=18)
        CS1 = ax1.contourf(xi,yi,zi1,levels, cmap=plt.get_cmap('Blues'),alpha=1,extend='both')
        CS2 = ax2.contourf(xi,yi,zi2,levels, cmap=plt.get_cmap('Blues'),alpha=1,extend='both')
        fig.colorbar(CS2, cax, orientation='horizontal')
        ax1.axvline(x=above_thresh_sigma[0], linewidth=2, color='k', linestyle='--')
        ax2.axvline(x=above_thresh_sigma[0], linewidth=2, color='k', linestyle='--')
        ax1.axhline(y=above_thresh_sigma[1], linewidth=2, color='k', linestyle='--')
        ax2.axhline(y=above_thresh_sigma[1], linewidth=2, color='k', linestyle='--')
        plt.savefig(path+'sim_precisions_and_recalls.png')
        plt.close()

    print("Best sigmas found:"+str(above_thresh_sigma[0])+', '+str(above_thresh_sigma[1]))
    return above_thresh_sigma[0],above_thresh_sigma[1]

def learning_curve(stable,variable,train, valid, precis_thresh, recall_thresh, rangeNums,path):
    validErr=[]
    valid_list=np.unique([x[0] for x in valid])
    output=open(path+'sigma_train.txt','w')
    trainErr=[]
    for num in rangeNums:
        if num>0:
            filename = open(path+"temp_sigma_data.txt", "w")
            filename.write('')
            filename.close()
            trainTMP=train[:num,:]
            train_list=np.unique([int(x[0]) for x in trainTMP])
            multiple_trials([[np.log10(float(x[1])), np.log10(float(x[2])), float(x[-1])] for x in trainTMP if float(x[1]) > 0 if float(x[2]) > 0],path+"temp_sigma_data.txt")
            data2=np.genfromtxt(path+'temp_sigma_data.txt', delimiter=' ')
            data=[[np.log10(float(trainTMP[n][1])),np.log10(float(trainTMP[n][2])),trainTMP[n][5],float(trainTMP[n][-1])] for n in range(num) if float(trainTMP[n][1]) > 0 if float(trainTMP[n][2]) > 0]
            best_sigma1, best_sigma2 = find_best_sigmas(precis_thresh,recall_thresh,data2,False,data,False,path)
            
            # Find the thresholds for a given sigma (in log space)
            sigcutx,paramx,range_x = generic_tools.get_sigcut([a[0] for a in data if a[3]==0.],best_sigma1)
            sigcuty,paramy,range_y = generic_tools.get_sigcut([a[1] for a in data if a[3]==0.],best_sigma2)

            # Calculate the training error
            fp=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FP'] for z in stable if (float(z[1])>=10.**sigcutx and float(z[2])>=10.**sigcuty) if int(z[0]) in train_list]) # False Positive
            fn=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FN'] for z in variable if (float(z[1])<10.**sigcutx or float(z[2])<10.**sigcuty) if int(z[0]) in train_list]) # False Negative
            trainErr.append(check_error(fp,fn,len(trainTMP)))

            # Caluculate the validation error
            fp=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FP'] for z in stable if (float(z[1])>=10.**sigcutx and float(z[2])>=10.**sigcuty) if int(z[0]) in valid_list]) # False Positive
            fn=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FN'] for z in variable if (float(z[1])<10.**sigcutx or float(z[2])<10.**sigcuty) if int(z[0]) in valid_list]) # False Negative
            validErr.append(check_error(fp,fn,len(valid)))
            output.write(str(num)+','+str(trainErr)+','+str(validErr))
    output.close()
     
    return  trainErr, validErr
        
def random_test(stable,variable,train, valid, precis_thresh, recall_thresh,path):
    multiple_trials([[np.log10(float(x[1])), np.log10(float(x[2])), float(x[-1])] for x in train if float(x[1]) > 0 if float(x[2]) > 0],"temp_sigma_data.txt")
    data2=np.genfromtxt('temp_sigma_data.txt', delimiter=' ')
    data=[[np.log10(float(train[n][1])),np.log10(float(train[n][2])),train[n][5],float(train[n][-1])] for n in range(len(train)) if float(train[n][1]) > 0 if float(train[n][2]) > 0]
    best_sigma1, best_sigma2 = find_best_sigmas(precis_thresh,recall_thresh,data2,False,data,False,path)

    # Find the thresholds for a given sigma (in log space)
    sigcutx,paramx,range_x = generic_tools.get_sigcut([a[0] for a in data if a[3]==0.],best_sigma1)
    sigcuty,paramy,range_y = generic_tools.get_sigcut([a[1] for a in data if a[3]==0.],best_sigma2)
            
    trainIDs=[str(int(x[0])) for x in train]
    validIDs=[str(int(x[0])) for x in valid]
    
    # Calculate the training error
    fp=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FP'] for z in stable if (float(z[1])>=10.**sigcutx and float(z[2])>=10.**sigcuty) if z[0] in trainIDs]) # False Positive
    fn=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FN'] for z in variable if (float(z[1])<10.**sigcutx or float(z[2])<10.**sigcuty) if z[0] in trainIDs]) # False Negative
    trainErr = check_error(fp,fn,len(train))

    # Caluculate the validation error
    fp=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FP'] for z in stable if (float(z[1])>=10.**sigcutx and float(z[2])>=10.**sigcuty) if z[0] in validIDs]) # False Positive
    fn=len([[z[0],float(z[1]),float(z[2]),float(z[3]),float(z[4]),'FN'] for z in variable if (float(z[1])<10.**sigcutx or float(z[2])<10.**sigcuty) if z[0] in validIDs]) # False Negative
    validErr = check_error(fp,fn,len(valid))
    
    #plotting_tools.plotLC(len(validErr), trainErr, validErr, path+'random', False, True, 'Trial number')
    return trainErr, validErr
        
def check_error(fp,fn,total):
    classif_err = float(fp+fn)/float(total)
    return classif_err

In [ ]:
#### SKIP CELL COUNTING (tools.py)

def SigmaFit(data):
    median = np.median(data)
    std_median = np.sqrt(np.mean([(i-median)**2. for i in data]))
    tmp_data = [a for a in data if a < 3.*std_median+median and a > median - 3.*std_median]
    param1 = norm.fit(tmp_data)
    param2 = norm.fit(data)
    return param1, param2

def extract_data(filename):
    # extract data in a csv file into a list
    info=[]
    data=open(filename,'r')
    for lines in data:
        if not lines.startswith("#"):
            lines=lines.rstrip().replace(" ", "")
            info.append(lines.split(','))
    data.close()
    return info

def write_data(filename,tmp):
    output = open(filename,'w')
    for line in tmp:
        output.write(str(line[0])+','+str(line[1])+','+str(line[2])+'\n')
    output.close()
    return

def read_src_lc(sources):
#
# Reads all the extracted source data and sorts them into unique sources with their lightcurves
#
    runcat=[] # A list comprising all the unique runcat source ids.
    new_source={} # A dictionary containing all the lightcurves, etc, for each unique source
    frequencies=[] # The frequencies in the dataset
    for a in range(len(sources)):
        new_runcat=sources[a][8]
        new_freq=int((float(sources[a][7])/1e6)+0.5) # observing frequency in MHz
        # check if it's a new runcat source and then either create a new entry or append
        if new_runcat not in runcat:
            runcat.append(new_runcat)
            new_source[new_runcat]=[sources[a]]
        else:
            new_source[new_runcat]=new_source[new_runcat]+[sources[a]]
        # If the observing frequency is new, append to the list
        if new_freq not in frequencies: 
            frequencies.append(new_freq)

    # return the list of observing frequencies and the full dictionary of source information
    return frequencies, new_source

def collate_trans_data(new_source,frequencies,transients):
#
# Using the data stored in the new_source dictionary and the transients list, store the transient and variability
# parameters for each unique source in the dataset.
#    
    trans_data=[]
    bands={}
    # sort the information in the transients list into a dictionary
    # transient id:[transient type, flux/max_rms, flux/min_rms, detection_thresh]
    transRuncat={x[5]:[x[2],float(x[1])/float(x[3]), float(x[1])/float(x[4]), x[0]] for x in transients}
    for freq in frequencies:
        for keys in new_source.keys():
            flux=[]
            flux_err=[]
            date=[]
            band=[]
            tmp=0.
            # Extract the different parameters for the source
            for b in range(len(new_source[keys])):
                if int((float(new_source[keys][b][7])/1e6)+0.5)==freq:
                    band.append(new_source[keys][b][0])
                    flux.append(float(new_source[keys][b][5]))
                    flux_err.append(float(new_source[keys][b][6]))
                    if tmp<int(new_source[keys][b][14]):
                        eta=float(new_source[keys][b][2])
                        V=float(new_source[keys][b][11])
                        N=float(new_source[keys][b][4])
                        tmp=int(new_source[keys][b][14])
                    ra=new_source[keys][b][-2]
                    dec=new_source[keys][b][-3]
            # if the source has been observed in the given observing frequency, extract the variability parameters
            # from the final observation at that frequency.
            if len(flux)!=0:
                bands[freq]=band
                ### Calculate the ratios...
                avg_flux_ratio = [x/(sum(flux)/len(flux)) for x in flux]
                ### Collate and store the transient parameters (these are across all the pipeline runs for the final figures)
                if keys in transRuncat.keys():
                # identify if this source is in the new source transient list and extract parameters
                    transType=transRuncat[keys][0]
                    min_sig=transRuncat[keys][1]
                    max_sig=transRuncat[keys][2]
                    detect_thresh=transRuncat[keys][3]
                else:
                # if not in the transient list, then insert standard parameters for non-transients
                    transType=2
                    min_sig=0
                    max_sig=0
                    detect_thresh=0
                # write out the key parameters for each source at each observing frequency
                trans_data.append([keys, eta, V, max(flux), max(avg_flux_ratio), freq, len(flux), ra, dec, transType, min_sig, max_sig, detect_thresh])
    print 'Number of transients in sample: '+str(len(trans_data))
    # Return the array of key parameters for each source
    return trans_data

In [ ]:
### SKIP CELL COUNTING (plotting)

def make_cmap(frequencies):
    cm = matplotlib.cm.get_cmap('hsv')
    col = [cm(1.*i/len(frequencies)) for i in range(len(frequencies))]
    return col

def gaussian_fit(data,param):
    range_data=np.linspace(min(data),max(data),1000)
    fit=norm.pdf(range_data,loc=param[0],scale=param[1])
    return range_data,fit

def make_bins(x):
    new_bins = density_estimation.bayesian_blocks(x)
    binsx = [new_bins[a] for a in range(len(new_bins)-1) if abs((new_bins[a+1]-new_bins[a])/new_bins[a])>0.05]
    binsx = binsx + [new_bins[-1]]
    return binsx

In [7]:
###FIND PRECISION AND RECALL FOR GIVEN THRESHOLDS

# Load the data and give appropriate labels
all_data = generic_tools.load_data(stableData,simulatedData)

# Load the simulations and only keep those with a full lightcurve and with variability parameters > 0
all_data=all_data.loc[(all_data['ttype'] == 2) & (all_data['V']>0.) & (all_data['eta']>0.)]

# put the training data into the format required for the training
train_data = all_data.apply(lambda row:[np.log10(row['eta']),np.log10(row['V']),row['variable'],row['label']],axis=1)
train_data = train_data.as_matrix()

# Obtain the training data if it doesn't already exist on disk (SLOW!)
if not os.path.exists(path+'sigma_data.txt'):
    filename = open(path+"sigma_data.txt", "w")
    filename.write('')
    filename.close()
    train_anomaly_detect.multiple_trials(train_data,path+"sigma_data.txt")
data2=np.genfromtxt(path+'sigma_data.txt', delimiter=' ')

# Using the training data, find the sigma threshold combination that best fits input thresholds
best_sigma1, best_sigma2 = train_anomaly_detect.find_best_sigmas(precis_thresh,recall_thresh,data2,tests,train_data,plots,path)
print 'sigma_(eta_nu)='+str(best_sigma1)+', sigma_(V_nu)='+str(best_sigma2)

# Find the eta and V thresholds for the data
sigcutx,paramx,range_x = generic_tools.get_sigcut([a[0] for a in train_data if a[2]==0.],best_sigma1)
sigcuty,paramy,range_y = generic_tools.get_sigcut([a[1] for a in train_data if a[2]==0.],best_sigma2)
print(r'Gaussian Fit $\eta$: '+str(round(10.**paramx[0],2))+'(+'+str(round((10.**(paramx[0]+paramx[1])-10.**paramx[0]),2))+' '+str(round((10.**(paramx[0]-paramx[1])-10.**paramx[0]),2))+')')
print(r'Gaussian Fit $V$: '+str(round(10.**paramy[0],2))+'(+'+str(round((10.**(paramy[0]+paramy[1])-10.**paramy[0]),2))+' '+str(round((10.**(paramy[0]-paramy[1])-10.**paramy[0]),2))+')')
print 'Eta_nu threshold='+str(10.**sigcutx)+', V_nu threshold='+str(10.**sigcuty)
threshx=10.**sigcutx
threshy=10.**sigcuty

# Calculate the false positives (FP), true negatives (TN), true positives (TP) and false negatives (FN)
all_data.loc[(((all_data['eta']<threshx) | (all_data['V']<threshy)) & (all_data['variable'] == 1)),'classified'] = 'FN'
all_data.loc[((all_data['eta']>=threshx) & (all_data['V']>=threshy) & (all_data['variable'] == 1)),'classified'] = 'TP'
all_data.loc[(((all_data['eta']<threshx) | (all_data['V']<threshy)) & (all_data['variable'] == 0)),'classified'] = 'TN'
all_data.loc[((all_data['eta']>=threshx) & (all_data['V']>=threshy) & (all_data['variable'] == 0)),'classified'] = 'FP'

# Find candidates
all_data.loc[(all_data['classified'] == 'FP')].to_csv(path+'AD_candidate_variables.csv',index=False)

# Calculate the precision and recall
precision, recall =  generic_tools.precision_and_recall(len(all_data.loc[(all_data['classified'] == 'TP')]),len(all_data.loc[(all_data['classified'] == 'FP')]),len(all_data.loc[(all_data['classified'] == 'FN')]))
print "Precision: "+str(precision)+", Recall: "+str(recall)

SyntaxError: invalid syntax (<ipython-input-7-a5ef82a5b27a>, line 21)

In [8]:
### TRAINING THE DATA


# Inputs and settings
precis_thresh = 0.99
recall_thresh = 0.95
lda=0.1
detection_threshold=8
path='ml_csv_files/'
stableData = path+'stable_sources.csv'
simulatedData = path+'sim_*_trans_data.csv'
anomaly = True
logistic = True
transSrc = True
# setting the options for the scipy optimise function
options = {'full_output': True, 'maxiter': 5000, 'ftol': 1e-4, 'maxfun': 5000, 'disp': True}

#load the data required
all_data = generic_tools.load_data(stableData,simulatedData)

#create the lambda curve for the logistic regreassion algorithm
MLtests.lambda_curve(all_data,lda,options,path)

# Create the learning curve for all three machine learning algorithms
MLtests.learning_curve(anomaly,logistic,transSrc,all_data,lda,options,precis_thresh,recall_thresh,path,detection_threshold)

# Create the repitition curve for all three machine learning algorithms
MLtests.repeat_curve(anomaly,logistic,transSrc,all_data,lda,options,precis_thresh,recall_thresh,path,detection_threshold)


NameError: name 'generic_tools' is not defined

In [10]:
# The input data and thresholds
tests = False
plots = False
path='ml_csv_files/'
stableData = path+'stable_sources.csv'
simulatedData = path+'sim_*_trans_data.csv'
detection_threshold = 8.

# Load the data and give appropriate labels
all_data = generic_tools.load_data(stableData,simulatedData)
# Identify all the new sources detected during the pipeline run
all_data2=all_data.loc[all_data['ttype'] != 2]
# Create a histogram of the detection thresholds assuming the source
# was found in the lowest RMS region of image
x=all_data2.loc[all_data2['variable'] == 0]
x2=all_data2.loc[all_data2['variable'] == 1]

# Find the best margins to maximise detection and reliability
best_data=all_data2[['minRmsSigma','variable']].as_matrix()
worst_data=all_data2[['maxRmsSigma','variable']].as_matrix()
best_plot_data, worst_plot_data, sigBest, sigWorst = train_sigma_margin.find_sigma_margin(best_data,worst_data, detection_threshold)

# Search and identify the optimal sigma margin for the best and
# worst parts of the image
sigWorst, sigBest = train_sigma_margin.plot_diagnostic(best_plot_data,worst_plot_data,path)

# Identify the ids of interesting transient candidates assuming they're in the
# worst part of the image
tmpData = all_data2.loc[(all_data2['maxRmsSigma'] >= sigWorst+detection_threshold) & (all_data2['variable']==0)]
tmpData.to_csv(path+'candidate_transients_worst_region.csv',index=False)

# Identify the ids of interesting transient candidates assuming they're in the
# best part of the image
tmpData = all_data2.loc[(all_data2['minRmsSigma'] >= sigBest+detection_threshold) & (all_data2['variable']==0)]
tmpData.to_csv(path+'candidate_transients_best_region.csv',index=False)

# And the best thresholds are:
print('Lowest RMS region threshold: '+str(sigBest+detection_threshold)+' sigma')
print('Highest RMS region threshold: '+str(sigWorst+detection_threshold)+' sigma')

NameError: name 'generic_tools' is not defined